In [1]:
from seisLM.model import lit_model
from seisLM.model import multidim_wav2vec2
import seisbench.data as sbd
import torch

In [2]:
data = sbd.STEAD()
waveforms = data.get_waveforms(1265656)
input_values = torch.Tensor(waveforms).unsqueeze(0).cuda()



2024-06-03 18:05:19,860 | seisbench | WARNING | Component order not specified, defaulting to 'ZNE'.


In [3]:
pretrained_model = lit_model.LitMultiDimWav2Vec2.load_from_checkpoint(
  '/home/liu0003/Desktop/projects/seisLM/saved_models/seisLM/42__2024-05-26-17h-04m-04s/checkpoints/epoch=3-step=218924.ckpt'
).model

In [4]:
config = pretrained_model.config
config.num_labels = 3
classification_model = multidim_wav2vec2.MultiDimWav2Vec2ForFrameClassification(
  config)

# Copy weights from pretrained model (if needed)
classification_model.wav2vec2.load_state_dict(pretrained_model.wav2vec2.state_dict())

classification_model.cuda();


In [5]:
outputs = classification_model(
  input_values=input_values,
  # interp_output_length
  # labels=one_hot_tensor.cuda() #torch.argmax(one_hot_tensor, 1).cuda()
)

In [6]:
outputs.shape

torch.Size([1, 3, 6000])

In [7]:
# labels = torch.argmax(outputs[0], 2)
# torch.argmax(labels, 1)

# outputs.shape
# config.getattr('interp_output_length', None)

In [3]:
from transformers import Wav2Vec2Config

model_config1 = Wav2Vec2Config.from_pretrained("facebook/wav2vec2-base")
model_config2 = Wav2Vec2Config.from_pretrained("patrickvonplaten/wav2vec2-base-v2")

/home/liu0003/miniconda3/envs/seisbench/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/liu0003/miniconda3/envs/seisbench/lib/python3.9/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [7]:
# model_config2.save_pretrained('config.json')

Wav2Vec2Config.from_pretrained('/home/liu0003/Desktop/projects/seisLM/configs/pretrain/pretrained_model_config.json')

Wav2Vec2Config {
  "activation_dropout": 0.0,
  "adapter_attn_dim": null,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    256,
    256
  ],
  "conv_kernel": [
    3,
    3
  ],
  "conv_stride": [
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "layer",
  "feat_proj_dropout": 0.0,
  "feat_quantizer_dropout": 0.0,
  "final_dropout": 0.0,
  "hidden_act": "gelu",
  "hidden_dropout": 0.0,
  "hidden_dropout_prob": 0.0,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 307

In [9]:
from seisLM.utils.project_path import gitdir

In [15]:
# bs = model_config.get('batch_size', None)
from ml_collections import config_dict

training_config = config_dict.ConfigDict()

# training_config.batch_size = 32
training_config.get('batch_size', None)
# print(bs)

In [17]:
import seisbench.data as sbd

dir(sbd)

['BenchmarkDataset',
 'Bucketer',
 'ChunkedDummyDataset',
 'DummyDataset',
 'ETHZ',
 'GEOFON',
 'GeometricBucketer',
 'ISC_EHB_DepthPhases',
 'InstanceCounts',
 'InstanceCountsCombined',
 'InstanceGM',
 'InstanceNoise',
 'Iquique',
 'LFEStacksCascadiaBostock2015',
 'LFEStacksMexicoFrank2014',
 'LFEStacksSanAndreasShelly2017',
 'LenDB',
 'MLAAPDE',
 'Meier2019JGR',
 'MultiWaveformDataset',
 'NEIC',
 'OBS',
 'OBST2024',
 'PNW',
 'PNWAccelerometers',
 'PNWExotic',
 'PNWNoise',
 'Ross2018GPD',
 'Ross2018JGRFM',
 'Ross2018JGRPick',
 'SCEDC',
 'STEAD',
 'TXED',
 'WaveformDataWriter',
 'WaveformDataset',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'base',
 'dummy',
 'ethz',
 'geofon',
 'instance',
 'iquique',
 'isc_ehb',
 'lendb',
 'lfe_stacks',
 'neic',
 'obs',
 'obst2024',
 'pnw',
 'scedc',
 'stead',
 'txed']